# Model params tuning (RF and ridge)

## RF

In [2]:
#- Tune parameters for RF
%run scripts/calibration/RF_tune.py #use this in terminal in different steps

In [ ]:
#- Pool and plot the results for RF
%run scripts/calibration/RF_post_tune.py

## Ridge

In [5]:
#- Tune for ridge
%run scripts/calibration/ridge_tune.py

Data shape: (11, 34) (n_samples_time_series*n_genes)
----Tuning for ctr iteration 0-----
stats: 34 genes 140 permts 10 threads
Running 60 samples randomly
Permutation-based multi threading


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



TypeError: 

In [ ]:
#- Pool and plot the results for ridge
%run scripts/calibration/ridge_post_tune.py

In [37]:
scores = [[np.ones(4), 2*np.ones(4), 3*np.ones(4)], [np.ones(4), 2*np.ones(4), 3*np.ones(4)]]
scores_pool = np.stack(scores,axis=2)
print('pool',scores_pool)
scores_mean = np.mean(scores_pool, axis=2)
print('mean',scores_mean)
params = ['p1','p2','p3','p4']
best_indices = np.argmax(scores_mean, axis=1)
print(best_indices)
best_scores = np.max(scores_mean, axis=1)
print(best_scores)
best_params = [params[i] for i in best_indices]

pool [[[1. 1.]
  [1. 1.]
  [1. 1.]
  [1. 1.]]

 [[2. 2.]
  [2. 2.]
  [2. 2.]
  [2. 2.]]

 [[3. 3.]
  [3. 3.]
  [3. 3.]
  [3. 3.]]]
mean [[1. 1. 1. 1.]
 [2. 2. 2. 2.]
 [3. 3. 3. 3.]]
[0 0 0]
[1. 2. 3.]


In [41]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
X = np.array([[1, 2], [3, 4],[3, 4]])
loo = LeaveOneOut()
print(loo.get_n_splits(X))

for i, (train_index, test_index) in enumerate(loo.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

3
Fold 0:
  Train: index=[1 2]
  Test:  index=[0]
Fold 1:
  Train: index=[0 2]
  Test:  index=[1]
Fold 2:
  Train: index=[0 1]
  Test:  index=[2]


# GRN 

## RF

In [ ]:
# run this one from terminal, as it needs to be run in seperate sessions (1000 runs)
%run scripts/GRN/RF_grn.py

In [ ]:
#pool the links and add oob scores to the df
%run scripts/GRN/RF_post_grn.py

## Ridge

In [1]:
%run scripts/GRN/ridge_grn.py

Data shape: (11, 34) (n_samples_time_series*n_genes)

network inference: train score, mean: 0.6720147745400069 std: 0.352471475046745

network inference: train score, mean: 0.5152218969285599 std: 0.36733856792142966
Number of string links: 60
Number of extracted links: 100
match:  7


## Portia

In [ ]:
%run scripts/GRN/portia_grn.py

## Plot distribution of mean weights

In [ ]:
from imports import *
importlib.reload(utils)
links_ctr_rf, links_sample_rf = retreive_grn('ctr','rf'),retreive_grn('mg','rf')
links_ctr_portia, links_sample_portia = retreive_grn('ctr','portia'),retreive_grn('mg','portia')
links_ctr_ridge, links_sample_ridge = retreive_grn('ctr','ridge'),retreive_grn('mg','ridge')
links_combined = [[links_ctr_rf, links_sample_rf],
               [links_ctr_portia, links_sample_portia], 
               [links_ctr_ridge, links_sample_ridge]]

#- normalize

links_combined_n = [[utils.Links.nomalize(item) for item in row] for row in links_combined]
labels = ['RF','Portia',
          'Ridge']
colors = ['lightblue','pink']
utils.Links.plot_mean_weights(links_combined_n, labels, colors)

In [8]:
aa = np.array([])
aa = np.append(aa,1, inplace=True)
aa

TypeError: _append_dispatcher() got an unexpected keyword argument 'inplace'

## Compare different methods: matching with string

In [ ]:
%run scripts/GRN_vs_string.py

## Filter

In [ ]:
%run scripts/filter_links.py

## Plot shortlisted weights

In [ ]:

def plot_weight(links, dist_key='WeightPool'):
    links_s = links.sort_values('Weight', ascending=False).reset_index(drop=True)
    
    top_n = 12
    nrows = 4
    ncols = 3
    fig, axes = plt.subplots(nrows, ncols, tight_layout=True, figsize=(ncols*3, nrows*2))
    for idx in range(top_n):
        i = int(idx/(nrows-1))
        j = idx%ncols
        ax = axes[i][j]
        ax.hist(links_s[dist_key][idx], bins=20, decay_coeff=0.5,
                        histtype='stepfilled', #'bar', 'barstacked', 'step', 'stepfilled'
                        color='lightgreen',
                        ec='black',
                        rwidth=.9
                       )
        ax.set_xlabel('Interaction strength')
        ax.set_ylabel('Model count')
#         title = links['Regulator'][idx]+'-->'+links['Target'][idx]
        title= idx
        ax.set_title(title)
        ax.set_ymargin(.15)
        ax.set_xmargin(.15)
def plot_pvalue(x):
    fig, ax = plt.subplots(1, 1, tight_layout=True, figsize=(3, 2))
    
    ax.hist(x, bins=20, decay_coeff=0.5,
                    histtype='barstacked', #'bar', 'barstacked', 'step', 'stepfilled'
                    ec='black',
                    rwidth=.9
                   )
    ax.set_xlabel('Interaction strength')
    ax.set_ylabel('Model count')
links_short_ctr_ridge = retreive_grn('ctr','ridge')

links_target = links_short_ctr_ridge
plot_weight(links_target, dist_key='WeightPool')
# print(links_short_ctr_ensemble)

## create edges and nodes

In [ ]:
links_short_ctr_portia = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN','links_short_ctr_portia.csv'))
links_short_sample_portia = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN','links_short_mg_portia.csv'))

def process_nodes_edges(study, links, scores=None):
    #- create nodes and edges
    nodes, edges = utils.convert_links_to_nodes_edges(links, protnames, scores)
    #- output edges and nodes   
    utils.read_write_nodes_edges(nodes=nodes, edges=edges, study=study, mode='write', OUTPUT_DIR=OUTPUT_DIR)
process_nodes_edges('ctr', links_short_ctr_portia)
process_nodes_edges('mg', links_short_sample_portia)

# Vester's sensitivity analysis. 
- AS: active sum. Sum along rows of the influence matrix and it indicates how much does a variable influence all the others.
- PS: passive sum. Its is the sum along columns of the influence matrix and it indicates how sensitive a variable is, how does it react to the influence of others
- Q: AS/PS -> how dominant
- P: AS.PS -> how participative a variable is
- Active: +Q
- Passive: -Q, -P
- Critical: +Q, +P
- Buffering: -Q, -P

In [ ]:
importlib.reload(utils)
#- retreive data: short links
method = 'portia'
links_ctr = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN',f'links_short_ctr_{method}.csv'))
links_sample = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN',f'links_short_mg_{method}.csv'))

oo_vsa_ctr = utils.VSA.analyse(links_ctr, protnames)
oo_vsa_sample = utils.VSA.analyse(links_sample, protnames)
#- plot 1: seperate windows for ctr and sample
utils.VSA.plot_ctr_vs_sample(oo_vsa_ctr,oo_vsa_sample, protnames, OUTPUT_DIR=OUTPUT_DIR)

## Role change & plot 

In [ ]:
#- read the data
importlib.reload(utils)
# - plot 2: change in the roles 
df_role_change = utils.VSA.role_change(oo_vsa_ctr, oo_vsa_sample, target_roles=[3])
df_role_change.to_csv(os.path.join(OUTPUT_DIR,'VSA','df_role_change.csv'),index=False)
utils.VSA.plot_role_change(df_role_change, OUTPUT_DIR=OUTPUT_DIR)

# Sensitivity analysis

## multiplicative gaussian noise

In [ ]:
importlib.reload(utils)
#- retreive data
df_role_change = pd.read_csv(os.path.join(OUTPUT_DIR,'VSA','df_role_change.csv'),index_col=False)
target_prots = df_role_change['Entry'].values.tolist() 

method = 'portia'
links_ctr = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN',f'links_short_ctr_{method}.csv'))
links_sample = pd.read_pickle(os.path.join(OUTPUT_DIR,'GRN',f'links_short_mg_{method}.csv'))
#- multiplicative gaussian noise
# utils.sensitivity_analysis.multiGaussNoise(links_ctr, links_sample, protnames, target_prots, OUTPUT_DIR)
#- additive gaussian noise
utils.sensitivity_analysis.addGaussNoise(links_ctr, links_sample, protnames, target_prots, OUTPUT_DIR)

# Visualize protein network 

In [ ]:
importlib.reload(utils)
with open(os.path.join(OUTPUT_DIR, 'postprocess/map_protname_genename.json')) as f:
    map_protname_genename = json.load(f)['map']
# preferred_names = [map_protname_genename[protname] for protname in protnames]
preferred_names =protnames
def visualize(study):
    #- retreive nodes and edges
    nodes, edges = utils.read_write_nodes_edges(study=study, mode='read', OUTPUT_DIR=OUTPUT_DIR)
    utils.visualize_network(nodes, edges, 'ctr', protnames, preferred_names, OUTPUT_DIR)
visualize('ctr')

In [ ]:
import igraph
igraph.drawing.shapes.DiamondDrawer()